<a href="https://colab.research.google.com/github/ducvktran/SpaceX-Rocket-Launch-Data-Dashboard/blob/main/SpaceX_Rocket_Launch_Data_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset used to make the dashboard contains some data on SpaceX rocket launches. The 'class' column indicates whether the launch was successful: class = 1 means the launch was successful, and class = 0 means the launch was not successful.

## Setting Up Libraries

In [1]:
!pip install pandas dash

import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 12.2 MB/s eta 0:00:00


<ipython-input-1-633dfcf4011b>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-633dfcf4011b>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


## Make the Dashboard

In [2]:
#Read the SpaceX launch data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

#Create a dash application
app = dash.Dash(__name__)

#App layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center',
                   'color': '#503D36',
                   'font-size': 40}),
    #Dropdown list for launch sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                    {'label': 'All sites', 'value': 'All'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                    ],
                 value='All',
                 placeholder='Select a launch site here',
                 searchable=True
                 ),
    html.Br(),
    #Pie chart to show launch outcomes for various launch sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P('Payload range (kg):'),
    #Slider for payload mass range
    dcc.RangeSlider(id='payload-slider',
                    min=0,
                    max=10000,
                    step=1000,
                    value=[min_payload, max_payload]
                    ),
    #Scatter chart to show correlation between payload mass and launch outcome
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ])

#Callback function for pie chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'All':
        data = spacex_df.groupby('Launch Site', as_index=False)['class'].sum()
        data.rename(columns={'class':'Success Count'}, inplace=True)
        fig = px.pie(data,
                     values='Success Count',
                     names='Launch Site',
                     title='Total Successful Launches for All Launch Sites')
        return fig
    else:
        data = pd.DataFrame({
            'Result': ['Success', 'Failure'],
            'Count': [spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['class'] == 1)].shape[0],
                      spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['class'] == 0)].shape[0]]
            })
        fig = px.pie(data,
                     values='Count',
                     names='Result',
                     title='Successes and Failures at {}'.format(entered_site))
        return fig

#Callback function for scatter chart
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload_range):
    data = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0])
                    &(spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    if entered_site == 'All':
        fig = px.scatter(data,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title='Payload Mass and Launch Outcome for All Sites')
        return fig
    else:
        plot_data = data[data['Launch Site'] == entered_site]
        fig = px.scatter(plot_data,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title='Payload Mass and Launch Outcome for {}'.format(entered_site))
        return fig

#Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>